<a href="https://colab.research.google.com/github/itazap/MIE424-FinalProject/blob/main/MIE424_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MIE424 Project

## 1. Load the Adult dataset

In [1]:
!git clone https://github.com/mlohaus/SearchFair.git
%cd SearchFair

Cloning into 'SearchFair'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 86 (delta 32), reused 74 (delta 24), pack-reused 0
Unpacking objects: 100% (86/86), done.
/content/SearchFair


In [2]:
import pandas as pd

In [3]:
# Load data into pandas DataFrame
dataset = pd.read_csv('data/adult/adult.csv')

# Drop fnlwgt, education, education-num, capital-gain, capital-loss as Lohaus et al do
dataset = dataset.drop(columns=['fnlwgt', 'education', 'capital-gain', 'capital-loss'])
dataset

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
0,39,State-gov,13,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K
1,50,Self-emp-not-inc,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K
2,38,Private,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K
3,53,Private,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K
4,28,Private,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,13,Divorced,Prof-specialty,Not-in-family,White,Female,36,United-States,<=50K
48838,64,?,9,Widowed,?,Other-relative,Black,Male,40,United-States,<=50K
48839,38,Private,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,50,United-States,<=50K
48840,44,Private,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,40,United-States,<=50K


In [4]:
def replaceWithOneHot(df, col_name):
    # Takes in a pandas dataframe and replaces column with name col_name
    # with multiple columns of its one-hot encoding
    one_hots = pd.get_dummies(dataset[col_name], prefix=col_name)
    df = df.drop(columns =[col_name])
    df = df.join(one_hots)
    return df 

## Onehot categorical variables
#for feature in ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'native-country']:
#    dataset = replaceWithOneHot(dataset, feature);

#Format below features in dataset to be binary based on Lohaus' get_real_data.py
dataset.loc[dataset['age'] > 37, 'age'] = 1
dataset.loc[dataset['age'] != 1, 'age'] = 0

dataset.loc[dataset['workclass'] == 'Private', 'workclass'] = 1
dataset.loc[dataset['workclass'] != 1, 'workclass'] = 0

dataset.loc[dataset['education-num'] == 9, 'education-num'] = 1
dataset.loc[dataset['education-num'] != 1, 'education-num'] = 0

dataset.loc[dataset['marital-status'] == "Married-civ-spouse", 'marital-status'] = 1
dataset.loc[dataset['marital-status'] != 1, 'marital-status'] = 0

dataset.loc[dataset['occupation'] == "Craft-repair", 'occupation'] = 1
dataset.loc[dataset['occupation'] != 1, 'occupation'] = 0

dataset.loc[dataset['relationship'] == "Not-in-family", 'relationship'] = 1
dataset.loc[dataset['relationship'] != 1, 'relationship'] = 0

dataset.loc[dataset['race'] == "White", 'race'] = 1
dataset.loc[dataset['race'] != 1, 'race'] = 0

dataset.loc[dataset['hours-per-week'] > 40, 'hours-per-week'] = 1
dataset.loc[dataset['hours-per-week'] != 1, 'hours-per-week'] = 0

dataset.loc[dataset['native-country'] == "United-States", 'native-country'] = 1
dataset.loc[dataset['native-country'] != 1, 'native-country'] = 0

#Replace 'Male' with 1 and 'Female' with 0 in sex column
dataset.loc[dataset['sex'] == 'Male', 'sex'] = 1
dataset.loc[dataset['sex'] == 'Female', 'sex'] = -1

#replace '>50K' with 1 and '<=50K' with 0 in income column
dataset.loc[dataset['income'] == '>50K', 'income'] = 1
dataset.loc[dataset['income'] == '<=50K', 'income'] = -1

dataset

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
0,1,0,0,0,0,1,1,1,0,1,-1
1,1,0,0,1,0,0,1,1,0,1,-1
2,1,1,1,0,0,1,1,1,0,1,-1
3,1,1,0,1,0,0,0,1,0,1,-1
4,0,1,0,1,0,0,0,-1,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...
48837,1,1,0,0,0,1,1,-1,0,1,-1
48838,1,0,1,0,0,0,0,1,0,1,-1
48839,1,1,0,1,0,0,1,1,1,1,-1
48840,1,1,0,0,0,0,0,1,0,1,-1


In [5]:
# Lohaus uses a random 10000 points for training, validation, and the rest for testing
# Since adult.csv is already shuffled, use the first 10000 rows as training and rest as testing

train_dataset = dataset.head(10000)
test_dataset = dataset.tail(len(dataset) - 10000)

## 2. Implement baseline models